Nhận xét:Policy iteration sẽ có thời gian chạy lâu hơn Value iteration vì nó cần phải thực hiện lặp lại các bước cập nhật chính sách và đánh giá giá trị cho đến khi chính sách ổn định. Trong khi đó, Value iteration chỉ cần lặp lại đánh giá giá trị cho đến khi giá trị hội tụ và sau đó sử dụng các giá trị này để xây dựng chính sách tối ưu trực tiếp. Do đó, Value iteration thường được ưu tiên sử dụng trong các môi trường có kích thước lớn hơn và cần tăng tốc hơn. Còn Policy iteration có thể cho kết quả ổn định và tối ưu hơn so với Value iteration.

## Define

In [59]:
import gym
import numpy as np
import time
from IPython import display

In [60]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [61]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [62]:
def value_iteration(env, max_iters=500, gamma=0.9):
    start_time=time.time()
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values,(time.time()-start_time)

In [63]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [64]:
def policy_extraction(env, v_values, gamma=0.9):
    # start_time=time.time()
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

In [65]:
def policy_iteration(env, max_iters=500, gamma=0.9):
    start_time=time.time()
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=int)
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        # Policy Evaluation
        for j in range(max_iters):
            prev_v_values = np.copy(v_values)

            # update the v-value for each state
            for state in range(env.observation_space.n):
                action = policy[state]
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                v_values[state] = q_value
            
            # check for convergence
            if np.all(np.isclose(v_values, prev_v_values)):
                # print(f'Policy evaluation converged at {j}-th iteration.')
                break

        # Policy Improvement
        policy_stable = True
        for state in range(env.observation_space.n):
            old_action = policy[state]

            q_values = []
            for action in range(env.action_space.n):
                q_value = 0
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * v_values[next_state])

                q_values.append(q_value)

            best_action = np.argmax(q_values)
            policy[state] = best_action

            if old_action != best_action:
                policy_stable = False

        if policy_stable:
            # print(f'Policy iteration converged at {i}-th iteration.')
            break  
    return policy,float(time.time()-start_time) 

## FrozenLake-v1

In [66]:
env = gym.make('FrozenLake-v1')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [67]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [68]:
env.observation_space.n

16

In [69]:
env.action_space.n

4

In [70]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 9)

In [71]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1)

(0.0, 6)

In [72]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2)

(0.0, 14)

In [73]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3)

(1.0, 16)

In [74]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [75]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 57/1000
Average number of steps: 12.31578947368421


In [76]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 106/1000
Average number of steps: 15.150943396226415


In [77]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 716/1000
Average number of steps: 37.37988826815643


In [78]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [79]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [80]:
np.all(v_values_1 >= v_values_0)

True

In [81]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [82]:
np.all(v_values_2 >= v_values_1)

True

In [83]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [84]:
np.all(v_values_3 >= v_values_2)

True

In [85]:
optimal_v_values,time_value = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [86]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [87]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

<ipython-input-64-c8abf9d624a5>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


In [88]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])

In [89]:
optimal_policy
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 731/1000
Average number of steps: 37.12175102599179


In [90]:
play_multiple_times(env, policy_1, 1000)

Number of successes: 59/1000
Average number of steps: 11.915254237288135


In [91]:
policy,time_pi= policy_iteration(env, max_iters=500, gamma=0.9)

In [92]:
play_multiple_times(env,policy, 1000)

Number of successes: 741/1000
Average number of steps: 36.666666666666664


In [93]:
time_pi<=time_value

False

## FrozenLake8x8-v1

In [114]:
env = gym.make('FrozenLake8x8-v1')

In [115]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [116]:
env.observation_space.n

64

In [117]:
env.action_space.n

4

In [118]:
optimal_v_values,time_value_8x8 = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 117-th iteration.


In [119]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

<ipython-input-64-c8abf9d624a5>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


In [120]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 708/1000
Average number of steps: 73.02824858757062


In [121]:
policy,time_pi_8x8= policy_iteration(env, max_iters=500, gamma=0.9)

In [122]:
play_multiple_times(env,policy, 1000)

Number of successes: 741/1000
Average number of steps: 73.97031039136303


In [123]:
time_pi_8x8<=time_value_8x8

True

## Taxi-v3

In [134]:
env = gym.make('Taxi-v3')

In [135]:
env.P[0][3] # Transition model

[(1.0, 0, -1, False)]

In [136]:
env.observation_space.n

500

In [137]:
env.action_space.n

6

In [138]:
optimal_v_values,time_value_v3 = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 116-th iteration.


In [139]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

<ipython-input-64-c8abf9d624a5>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  policy = np.zeros(env.observation_space.n, dtype=np.int)


In [140]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 1000/1000
Average number of steps: 12.993


In [141]:
policy,time_pi_v3= policy_iteration(env, max_iters=500, gamma=0.9)

In [142]:
play_multiple_times(env,policy, 1000)

Number of successes: 1000/1000
Average number of steps: 13.008


In [143]:
time_pi_v3<=time_value_v3

True